In [ ]:
import sys
sys.path.append('..')
import numpy as np
import math
import matplotlib.pylab as plt
import rti_msr_utils as rti



In [ ]:
num_CIR_samples = 50  # number of logged CIR samples
CIR_samp_period = 1/(2*(4*124.8e6))  # (s) sampling period for CIR measurement
anchor_UWB_Ids = [0, 1, 2, 3, 4]  # device Ids
SpeedofLight = 299792458  # (m/s) speed of light

In [ ]:
anchor_pair_list = []
for i in range(0,5):
    for j in range(i+1,5):
        anchor_pair_list.append((i,j))

![image](..\pic\room1.png)

In [ ]:
# room 1
pos = [(2.28,0.9),(0,1.55),(5.09,6.52),(5.09,2.2),(0.2,5.69)]
room_length = 5.09
room_width = 10.02
rel_x = 1.25
rel_y = 1.98
# x_monitored = 2.45
# y_monitored = 4.40
x_monitored = room_length
y_monitored = room_width

In [ ]:
mirrored_positions = rti.generate_mirror_pos(pos, room_length, room_width)
print(mirrored_positions)

In [ ]:
# visualize the room
rti.visualize_room(room_length, room_width, pos, mirrored_positions, rel_x, rel_y)

In [ ]:
mpc_position = rti.generate_mpc_position(anchor_pair_list,mirrored_positions)

In [ ]:
# remove item in mpc_position that differ less than 0.01
for item in mpc_position:
    array = mpc_position[item]
    array = np.sort(array)
    mask = np.zeros(len(array),dtype=bool)
    for i in range(1,len(array)):
        if array[i] - array[i-1] < 0.01:
            mask[i] = True
        if array[i] > rti.maximum_distance:
            mask[i] = True

    mpc_position[item] = np.delete(array,mask)

![image](..\pic\weight.png)

![image](..\pic\parameters.png)

In [ ]:
prm_lambda = 0.01 
pixel_width = 0.1
room_x_len = math.floor(x_monitored/pixel_width) #romm_x_len = math.floor(room_length/pixel_width)
room_y_len = math.floor(y_monitored/pixel_width) #romm_y_len = math.floor(room_width/pixel_width)
pixel_num_j = room_x_len * room_y_len #left bottom is origin point
num_of_mirror = 5

In [ ]:
weight_matrix = rti.generate_weight_matrix(anchor_pair_list,mpc_position,mirrored_positions,room_x_len,room_y_len,pixel_width,num_of_mirror,prm_lambda)

In [ ]:
anchor_pair = (2,3) # Replace with an actual pair from your list
distance = 2.0  # Replace with an actual distance from your mpc_position dictionary
rti.plot_weight_matrix(weight_matrix, anchor_pair, round(mpc_position[anchor_pair][2],2), room_x_len, room_y_len)

In [ ]:
regularization_matrix_c = rti.generate_regularization_matrix(room_x_len, room_y_len, pixel_width)

In [ ]:
rti.plot_regularization_matrix(regularization_matrix_c)

In [ ]:
file_path = "../../data/uwb2/uwb2_exp001.csv"
uwb2_exp001, cir, tx_id_uwb, rx_id_uwb, rx_pream_count, fp, rx_lvl, time_stamp, ground_x, ground_y = rti.load_csv_data(file_path, num_CIR_samples)

cir_up = rti.upsample_and_align_cir(cir, fp, freq_s_ratio=64)
grouped_data = rti.group_data_by_tx_rx(tx_id_uwb, rx_id_uwb, cir, cir_up, rx_pream_count, fp, rx_lvl, time_stamp, ground_x, ground_y)


In [ ]:
anchor_pair = anchor_pair_list[9]
rti.plot_cir_lines_with_mpc(grouped_data[anchor_pair]["cir_up"], anchor_pair, mpc_position, 0, 10)

In [ ]:
mpc_mag_idle = {}
for anchor_pair in anchor_pair_list:
    mpc_mag_idle[anchor_pair] = rti.cal_mag_multipath_component(anchor_pair, mpc_position, grouped_data, 
                                0, len(grouped_data[anchor_pair]["cir_up"]), 64)

In [ ]:
file_path = "../../data/uwb2/uwb2_exp001.csv"
uwb2_exp001, cir, tx_id_uwb, rx_id_uwb, rx_pream_count, fp, rx_lvl, time_stamp, ground_x, ground_y = rti.load_csv_data(file_path, num_CIR_samples)
cir_up = rti.upsample_and_align_cir(cir, fp, freq_s_ratio=64)
grouped_data_001 = rti.group_data_by_tx_rx(tx_id_uwb, rx_id_uwb, cir, cir_up, rx_pream_count, fp, rx_lvl, time_stamp, ground_x, ground_y)

In [ ]:
file_path = "../../data/uwb2/uwb2_exp002.csv"
uwb2_exp001, cir, tx_id_uwb, rx_id_uwb, rx_pream_count, fp, rx_lvl, time_stamp, ground_x, ground_y = rti.load_csv_data(file_path, num_CIR_samples, tag_name_left="tag4422_", tag_name_right="tag89b3_")
cir_up = rti.upsample_and_align_cir(cir, fp, freq_s_ratio=64)
grouped_data = rti.group_data_by_tx_rx(tx_id_uwb, rx_id_uwb, cir, cir_up, rx_pream_count, fp, rx_lvl, time_stamp, ground_x, ground_y)

In [ ]:
from bisect import bisect_left
def cal_mag_multipath_component(anchor_pair, mpc_position, grouped_data, start, end, ratio):
    positions = mpc_position[anchor_pair]

    cir_up = grouped_data[anchor_pair]["cir_up"]
    cir_up_mean = np.mean(np.abs(cir_up[start:end]),axis=0)
    pream_count = np.mean(grouped_data[anchor_pair]["rx_pream_count"][start:end])
    recei_power_level = np.mean(grouped_data[anchor_pair]["rx_lvl"][start:end])

    # if cir_up are empty, continue
    if len(cir_up) == 0:
        return []


    mpc_mag = []
    for pos in positions:
        pos = int(pos / 0.3 * ratio) + 4*ratio
        power_of_path = 10*math.log10((cir_up_mean[pos] ** 2 + 
                            cir_up_mean[pos - ratio] ** 2 + 
                            cir_up_mean[pos + ratio] ** 2)/(pream_count ** 2)) - 121.74
        power_of_path = power_of_path - recei_power_level
        mpc_mag.append(power_of_path)
    return mpc_mag

def find_timestamp_indices(ts, start_time, end_time):
    start_idx = bisect_left(ts, start_time)
    end_idx = bisect_left(ts, end_time, lo=start_idx)

    return start_idx, end_idx

def cal_z(anchor_pair_list, mpc_mag_idle, grouped_data, mpc_position, start_time, end_time):
    z = []
    missing_mpc = []  # List to store anchor_pair with missing mpc_mag_pair

    for anchor_pair in anchor_pair_list:
        mpc_mag_pair = mpc_mag_idle.get(anchor_pair, [])
        
        if not mpc_mag_pair:
            missing_mpc.append(anchor_pair)  # Save the anchor_pair for later processing
            continue

        data_pair = grouped_data.get(anchor_pair, {})
        ts = data_pair.get("time_stamp", [])

        start_idx, end_idx = find_timestamp_indices(ts, start_time, end_time)
        mpc_map = cal_mag_multipath_component(anchor_pair, mpc_position, grouped_data, start_idx, end_idx, 64)
        z.append(np.abs(np.array(mpc_map) - np.array(mpc_mag_pair)))

    return z, missing_mpc  # Return both lists

def erase_elements(z, matrix, erase_indices):
    # Erase elements from z
    new_z = np.delete(z, erase_indices)

    # Erase rows from matrix
    new_matrix = np.delete(matrix, erase_indices, axis=0)
    
    return new_z, new_matrix

In [ ]:
start = 54493
end = start + 0.2

z, missing_mpc = cal_z(anchor_pair_list, mpc_mag_idle, grouped_data, mpc_position, start, end)
weight_matrix_np = rti.convert_weight_matrix_to_numpy(weight_matrix,anchor_pair_list,mpc_position,pixel_num_j, missing_mpc)

# Flattening the list of arrays and concatenating them into a single 1D array
z_flattened = np.concatenate([np.array(array).flatten() for array in z])

# z_flattened,weight_matrix_np =  erase_elements(z_flattened,weight_matrix_np,[17,18,19,20,21,22,23,24])

rti_matrix = rti.calculate_rti_matrix(weight_matrix_np, regularization_matrix_c)

In [ ]:
z,anchor_pair_list

In [ ]:



# for i in range(len(z_flattened)):
#     z_flattened[i] = 0

# Keeping track of the original shapes of the arrays
original_shapes = [array.shape for array in z]


res = np.matmul(rti_matrix,z_flattened)
plt.imshow((res).reshape(room_y_len,room_x_len), origin='lower')
plt.colorbar()

anchor_pair = (0,4)
start_index, end_index = rti.find_timestamp_indices(grouped_data[anchor_pair]["time_stamp"], start, end)
x_pos = grouped_data[anchor_pair_list[0]]["ground_x"][start_index:end_index]
y_pos = grouped_data[anchor_pair_list[0]]["ground_y"][start_index:end_index]
plt.plot(x_pos[0] / 0.1, y_pos[0] / 0.1, c='red', marker='o')

for p in pos:
    plt.plot(p[0]/0.1,p[1]/0.1, c='black', marker='o')

In [ ]:
z

In [ ]:
weight_matrix_np.shape,

In [ ]:
anchor_pair = (2,3)
start_index, end_index = rti.find_timestamp_indices(grouped_data[anchor_pair]["time_stamp"], start, end)

rti.plot_overlayed_weight_matrices(weight_matrix, anchor_pair, room_x_len, room_y_len, grouped_data[anchor_pair]["ground_x"][start_index:end_index], grouped_data[anchor_pair]["ground_y"][start_index:end_index])
# # statistics 
# rti.plot_cir_lines_with_mpc_mean(grouped_data_001[anchor_pair]["cir_up"], anchor_pair, mpc_position, 0, len(grouped_data_001[anchor_pair]["cir_up"]))
# # real time
# rti.plot_cir_lines_with_mpc_mean(grouped_data[anchor_pair]["cir_up"], anchor_pair, mpc_position, start, end)

# combined
rti.plot_2_cir_lines_with_mpc_mean(grouped_data_001[anchor_pair]["cir_up"], grouped_data[anchor_pair]["cir_up"], anchor_pair, mpc_position, 0, len(grouped_data_001[anchor_pair]["cir_up"]), start_index, end_index)


# anchor position
anchor_pos_0 = mirrored_positions[anchor_pair[0]][0]
anchor_pos_1 = mirrored_positions[anchor_pair[1]][0]

# mean postion of tag   
tag_pos = (
    np.mean(grouped_data[anchor_pair]["ground_x"][start_index:end_index]),
    np.mean(grouped_data[anchor_pair]["ground_y"][start_index:end_index]))

# distance to anchor
rti.cal_distance(tag_pos, anchor_pos_0) + rti.cal_distance(tag_pos, anchor_pos_1) - rti.cal_distance(anchor_pos_0, anchor_pos_1)

In [ ]:
z_flattened,original_shapes,anchor_pair_list